In [29]:


import TetrisUtils
from test_TetrisUtils import board1,board2,board3

In [30]:
def speed_aggregate_height():
    for i in range(10000):
        height = TetrisUtils.get_aggregate_height(board1)
        height = TetrisUtils.get_aggregate_height(board2)
        height = TetrisUtils.get_aggregate_height(board3)

def speed_hollow_column_count():
    for i in range(10000):
        height = TetrisUtils.get_hollow_column_count(board1)
        height = TetrisUtils.get_hollow_column_count(board2)
        height = TetrisUtils.get_hollow_column_count(board3)
def speed_row_transition():
    for i in range(10000):
        height = TetrisUtils.get_row_transition(board1)
        height = TetrisUtils.get_row_transition(board2)
        height = TetrisUtils.get_row_transition(board3)
def speed_col_transition():
    for i in range(10000):
        height = TetrisUtils.get_col_transition(board1)
        height = TetrisUtils.get_col_transition(board2)
        height = TetrisUtils.get_col_transition(board3)
print(TetrisUtils.get_aggregate_height(board1))
print(TetrisUtils.get_aggregate_height(board2))
print(TetrisUtils.get_aggregate_height(board3))

10
20
24


In [31]:
%time speed_aggregate_height()
%time  speed_hollow_column_count()
%time speed_row_transition()
%time speed_col_transition()

CPU times: total: 203 ms
Wall time: 182 ms
CPU times: total: 156 ms
Wall time: 153 ms
CPU times: total: 109 ms
Wall time: 115 ms
CPU times: total: 219 ms
Wall time: 212 ms


# Numpy version

In [32]:
import numpy as np

In [33]:
board1 = np.array(board1)
board2 = np.array(board2)
board3 = np.array(board3)
board1



array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0],
       [1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0]])

In [34]:
board1.nonzero()
boolean_board = board1.astype(bool)

height = board1.shape[0] - np.argmax(boolean_board,axis=0)
#board1.shape[0]
height

height[height==board1.shape[0]] = 0
height
np.sum(height)
#height = np.where(height!=board1.shape[0])
#height
#sum(np.count_nonzero(board1,axis=0))

10

In [35]:
board1[5,2]

1

In [36]:
def aggregate_height(board):
    board_len = board.shape[0]
    boolean_board = board.astype(bool)
    height = board_len - np.argmax(boolean_board,axis=0)
    height[height==board_len] = 0
    return np.sum(height)

print(aggregate_height(board1))
print(aggregate_height(board2))
print(aggregate_height(board3))

10
20
24


In [37]:
def numpy_aggregate_height():
    for i in range(10000):
        height = aggregate_height(board1)
        height = aggregate_height(board2)
        height = aggregate_height(board3)

In [38]:
%time numpy_aggregate_height()

CPU times: total: 266 ms
Wall time: 256 ms


# Speedy version

In [39]:
from numba import jit
from numba.typed import List

In [40]:
@jit(nopython=True)
def get_aggregate_height(board: List) -> int:
    GRID_COL_COUNT = len(board[0])
    GRID_ROW_COUNT = len(board)
    aggregate_height = 0
    column_checked = List()
    for row in range(GRID_ROW_COUNT):
        for col in range(GRID_COL_COUNT):
            if not (col in column_checked) and board[row][col] != 0:
                height = len(board) - row
                aggregate_height += height
                column_checked.append(col)

    return aggregate_height

board1 = List(board1)
board2 = List(board2)
board3 = List(board3)
height = get_aggregate_height(board1)

In [41]:
def loop_speedy():
    for i in range(10000):
        height = get_aggregate_height(board1)
        height = get_aggregate_height(board2)
        height = get_aggregate_height(board3)

In [42]:
%time loop_speedy()

CPU times: total: 62.5 ms
Wall time: 64.1 ms
